In [1]:
# Copyright 2020 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

<img src="http://developer.download.nvidia.com/compute/machine-learning/frameworks/nvidia_logo.png" style="width: 90px; float: right;">

# NVTabular demo on Rossmann data - PyTorch

## Overview

NVTabular is a feature engineering and preprocessing library for tabular data designed to quickly and easily manipulate terabyte scale datasets used to train deep learning based recommender systems.  It provides a high level abstraction to simplify code and accelerates computation on the GPU using the RAPIDS cuDF library.

### Learning objectives

In the previous notebooks ([rossmann-store-sales-preproc.ipynb](https://github.com/NVIDIA/NVTabular/blob/main/examples/rossmann/rossmann-store-sales-preproc.ipynb) and [rossmann-store-sales-feature-engineering.ipynb](https://github.com/NVIDIA/NVTabular/blob/main/examples/rossmann/rossmann-store-sales-feature-engineering.ipynb)), we downloaded, preprocessed and created features for the dataset. Now, we are ready to train our deep learning model on the dataset. In this notebook, we use **PyTorch** with the NVTabular data loader for PyTorch to accelereate the training pipeline.

In [2]:
import os
import math
import nvtabular as nvt
import glob

## Loading NVTabular workflow
This time, we only need to define our data directories. We can load the data schema from the NVTabular workflow.

In [3]:
DATA_DIR = os.environ.get("OUTPUT_DATA_DIR", "./data")
INPUT_DATA_DIR = os.environ.get("INPUT_DATA_DIR", "./data")
PREPROCESS_DIR = os.path.join(INPUT_DATA_DIR, 'ross_pre/')
PREPROCESS_DIR_TRAIN = os.path.join(PREPROCESS_DIR, 'train')
PREPROCESS_DIR_VALID = os.path.join(PREPROCESS_DIR, 'valid')

What files are available to train on in our directories?

In [4]:
!ls $PREPROCESS_DIR

stats_and_workflow  train  valid


In [5]:
!ls $PREPROCESS_DIR_TRAIN

0.7fcd6d8270764d4990ad7734342596f2.parquet  _file_list.txt  _metadata.json
1.461032071b2343fd8477bf5b5dc18545.parquet  _metadata


In [6]:
!ls $PREPROCESS_DIR_VALID

_metadata  part.0.parquet


To load a saved NVTabular workflow, we need to initalize a NVTabular workflow, first.

In [7]:
# Note, we can initialize it with an empty schema
proc = nvt.Workflow(
    cat_names=[],
    cont_names=[],
    label_name=[]
)
proc.load_stats(os.path.join(PREPROCESS_DIR, 'stats_and_workflow'))

We extract the categorical, continuous and label column names from the NVTabular workflow.

In [8]:
CATEGORICAL_COLUMNS = proc.columns_ctx['final']['cols']['categorical']
CONTINUOUS_COLUMNS = proc.columns_ctx['final']['cols']['continuous']
LABEL_COLUMNS = proc.columns_ctx['final']['cols']['label']

COLUMNS = CATEGORICAL_COLUMNS + CONTINUOUS_COLUMNS + LABEL_COLUMNS

We load the statistics for embedding tables of our neural network. The following shows the cardinality of each categorical variable along with its associated embedding size. Each entry is of the form `(cardinality, embedding_size)`.

In [9]:
EMBEDDING_TABLE_SHAPES = nvt.ops.get_embedding_sizes(proc)
EMBEDDING_TABLE_SHAPES

{'Assortment': (4, 3),
 'CompetitionMonthsOpen': (26, 10),
 'CompetitionOpenSinceYear': (24, 9),
 'Day': (32, 11),
 'DayOfWeek': (8, 5),
 'Events': (22, 9),
 'Month': (13, 7),
 'Promo2SinceYear': (9, 5),
 'Promo2Weeks': (27, 10),
 'PromoInterval': (4, 3),
 'Promo_bw': (7, 5),
 'Promo_fw': (7, 5),
 'SchoolHoliday_bw': (9, 5),
 'SchoolHoliday_fw': (9, 5),
 'State': (13, 7),
 'StateHoliday': (3, 3),
 'StateHoliday_bw': (4, 3),
 'StateHoliday_fw': (4, 3),
 'Store': (1116, 16),
 'StoreType': (5, 4),
 'Week': (53, 15),
 'Year': (4, 3)}

In [10]:
# Note, we can initialize it with an empty schema
proc = nvt.Workflow(
    cat_names=CATEGORICAL_COLUMNS,
    cont_names=CONTINUOUS_COLUMNS,
    label_name=LABEL_COLUMNS
)

In [11]:
proc.load_stats(os.path.join(PREPROCESS_DIR, 'stats_and_workflow'))
EMBEDDING_TABLE_SHAPES = nvt.ops.get_embedding_sizes(proc)

## Training a Network

Now that our data is preprocessed and saved out, we can leverage `dataset`s to read through the preprocessed parquet files in an online fashion to train neural networks.

We'll start by setting some universal hyperparameters for our model and optimizer. These settings will be the same across all of the frameworks that we explore in the different notebooks.

In [12]:
EMBEDDING_DROPOUT_RATE = 0.04
DROPOUT_RATES = [0.001, 0.01]
HIDDEN_DIMS = [1000, 500]
BATCH_SIZE = 65536
LEARNING_RATE = 0.001
EPOCHS = 25

# TODO: Calculate on the fly rather than recalling from previous analysis.
MAX_SALES_IN_TRAINING_SET = 38722.0
MAX_LOG_SALES_PREDICTION = 1.2 * math.log(MAX_SALES_IN_TRAINING_SET + 1.0)

TRAIN_PATHS = sorted(glob.glob(os.path.join(PREPROCESS_DIR_TRAIN, '*.parquet')))
VALID_PATHS = sorted(glob.glob(os.path.join(PREPROCESS_DIR_VALID, '*.parquet')))

## PyTorch<a id="PyTorch"></a>


### PyTorch: Preparing Datasets

In [13]:
import torch
from nvtabular.loader.torch import TorchAsyncItr, DLDataLoader
from nvtabular.framework_utils.torch.models import Model
from nvtabular.framework_utils.torch.utils import process_epoch

# TensorItrDataset returns a single batch of x_cat, x_cont, y.
collate_fn = lambda x: x

train_dataset = TorchAsyncItr(nvt.Dataset(TRAIN_PATHS), batch_size=BATCH_SIZE, cats=CATEGORICAL_COLUMNS, conts=CONTINUOUS_COLUMNS, labels=LABEL_COLUMNS)
train_loader = DLDataLoader(train_dataset, batch_size=None, collate_fn=collate_fn, pin_memory=False, num_workers=0)

valid_dataset = TorchAsyncItr(nvt.Dataset(VALID_PATHS), batch_size=BATCH_SIZE, cats=CATEGORICAL_COLUMNS, conts=CONTINUOUS_COLUMNS, labels=LABEL_COLUMNS)
valid_loader = DLDataLoader(valid_dataset, batch_size=None, collate_fn=collate_fn, pin_memory=False, num_workers=0)

### PyTorch: Defining a Model

In [14]:
model = Model(
    embedding_table_shapes=EMBEDDING_TABLE_SHAPES,
    num_continuous=len(CONTINUOUS_COLUMNS),
    emb_dropout=EMBEDDING_DROPOUT_RATE,
    layer_hidden_dims=HIDDEN_DIMS,
    layer_dropout_rates=DROPOUT_RATES,
    max_output=MAX_LOG_SALES_PREDICTION
).to('cuda')

### PyTorch: Training

In [15]:
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [16]:
def rmspe_func(y_pred, y):
    "Return y_pred and y to non-log space and compute RMSPE"
    y_pred, y = torch.exp(y_pred) - 1, torch.exp(y) - 1
    pct_var = (y_pred - y) / y
    return (pct_var**2).mean().pow(0.5)

In [17]:
%%time

for epoch in range(EPOCHS):
    train_loss, y_pred, y = process_epoch(train_loader, model, train=True, optimizer=optimizer)
    train_rmspe = rmspe_func(y_pred, y)
    valid_loss, y_pred, y = process_epoch(valid_loader, model, train=False)
    valid_rmspe = rmspe_func(y_pred, y)
    print(f'Epoch {epoch:02d}. Train loss: {train_loss:.4f}. Train RMSPE: {train_rmspe:.4f}. Valid loss: {valid_loss:.4f}. Valid RMSPE: {valid_rmspe:.4f}.')

Total batches: 12
Total batches: 0
Epoch 00. Train loss: 8.9780. Train RMSPE: inf. Valid loss: 3.8806. Valid RMSPE: 0.8406.
Total batches: 12
Total batches: 0
Epoch 01. Train loss: 3.9594. Train RMSPE: 0.8126. Valid loss: 3.3898. Valid RMSPE: 0.8194.
Total batches: 12
Total batches: 0
Epoch 02. Train loss: 2.5771. Train RMSPE: 0.7610. Valid loss: 1.8790. Valid RMSPE: 0.7111.
Total batches: 12
Total batches: 0
Epoch 03. Train loss: 1.3235. Train RMSPE: 0.6399. Valid loss: 0.8465. Valid RMSPE: 0.5532.
Total batches: 12
Total batches: 0
Epoch 04. Train loss: 0.4674. Train RMSPE: 0.5101. Valid loss: 0.2422. Valid RMSPE: 0.4450.
Total batches: 12
Total batches: 0
Epoch 05. Train loss: 0.2244. Train RMSPE: 0.6031. Valid loss: 0.2010. Valid RMSPE: 0.5736.
Total batches: 12
Total batches: 0
Epoch 06. Train loss: 0.2044. Train RMSPE: 0.6405. Valid loss: 0.1817. Valid RMSPE: 0.5311.
Total batches: 12
Total batches: 0
Epoch 07. Train loss: 0.1788. Train RMSPE: 0.5585. Valid loss: 0.1588. Valid RM